# XML Parser for Extracting JQA Subject Headings



In [1]:
# Import necessary libraries.
import re, glob, csv, sys, os
import pandas as pd
import xml.etree.ElementTree as ET

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/SemanticData/"

# Gather all .xml files using glob.
list_of_files = glob.glob(abs_dir + "Data/JQA_papers/*/*.xml")

## Define Functions

In [14]:
'''
Arguments of Functions:

    namespace:

    ancestor:
    
    xpath_as_string:
    
    attrib_val_str:
    
'''

# Read in file and get root of XML tree.
def get_root(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root


# Get namespace of individual file from root element.
def get_namespace(root):
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    return ns


# Get document id.
def get_document_id(ancestor, attrib_val_str):
    doc_id = ancestor.get(attrib_val_str)
    return doc_id


# Get date of document.
def get_date_from_attrValue(ancestor, xpath_as_string, attrib_val_str, namespace):
    date = ancestor.find(xpath_as_string, namespace).get(attrib_val_str)
    return date

# Get subject heading from document.
def get_subject_from_attrValue(ancestor, xpath_as_string, namespace):
    subject_list = []
    for elem in ancestor.findall(xpath_as_string, namespace):
        subject = ''.join(ET.tostring(elem, encoding='unicode', method='text'))
        subject_list.append(re.sub(r'\s+', ' ', subject))
#     Return a string object of 'list' to be written to output file. Can be split later.
    return ','.join(subject_list)

    
# Get plain text of every element (designated by first argument).
def get_textContent(ancestor, xpath_as_string, namespace):
    text_list = []
    for elem in ancestor.findall(xpath_as_string, namespace):
        text = ''.join(ET.tostring(elem, encoding='unicode', method='text'))

#         Add text (cleaned of additional whitespace) to text_list.
        text_list.append(re.sub(r'\s+', ' ', text))

#     Return concetanate text list.
    return ' '.join(text_list)

## Declare Variables

In [15]:
# Declare regex to simplify file paths below
regex = re.compile(r'.*/\d{4}/(.*)')

# Declare document level of file. Requires root starting point ('.').
doc_as_xpath = './/ns:div/[@type="entry"]'

# Declare date element of each document.
date_path = './ns:bibl/ns:date/[@when]'

# Declare subject elements in each document.
subject_path = './/ns:bibl/ns:note[@type="subject"]'

# Declare text level within each document.
text_path = './ns:div/[@type="docbody"]/ns:p'

## Parse Documents

In [16]:
%%time

# Open/Create file to write contents.
with open(abs_dir + 'Output/ParsedXML/JQA_Subjects-dataframe.txt', 'w') as outFile:
    
#     Write headers for table.
    outFile.write('file' + '\t' + 'entry' + '\t' + 'date' + '\t' + \
                  'subject' + '\t' + 'text' + '\n')
    
#     Loop through each file within a directory.
    for file in list_of_files:
        
#         Call functions to create necessary variables and grab content.
        root = get_root(file)
        ns = get_namespace(root)

        for eachDoc in root.findall(doc_as_xpath, ns):
#             Call functions.
            entry = get_document_id(eachDoc, '{http://www.w3.org/XML/1998/namespace}id')
            date = get_date_from_attrValue(eachDoc, date_path, 'when', ns)
            subject = get_subject_from_attrValue(eachDoc, subject_path, ns)
            text = get_textContent(eachDoc, text_path, ns)
            
#             Write results in tab-separated format.
            outFile.write(str(regex.search(file).groups()) + '\t' +  entry + \
                          '\t' + date + '\t' + subject + '\t' + text + '\n')

CPU times: user 576 ms, sys: 14.7 ms, total: 590 ms
Wall time: 597 ms


## Import Dataframe

In [17]:
dataframe = pd.read_csv(abs_dir + 'Output/ParsedXML/JQA_Subjects-dataframe.txt',
                        sep = '\t')

dataframe

,file,entry,date,subject,text
0,"('JQADiaries-v49-1825-01-p795.xml',)",jqadiaries-v49-1825-01-01,1825-01-01,NaN,"1. VI:30. H. Humphreys here, for Methodist Chu..."
1,"('JQADiaries-v49-1825-01-p795.xml',)",jqadiaries-v49-1825-01-02,1825-01-02,NaN,2. VII:15— Heard Lynde at the Capitol—late. Ca...
2,"('JQADiaries-v49-1825-01-p795.xml',)",jqadiaries-v49-1825-01-03,1825-01-03,NaN,3. VII. I called at M. Van-Buren’s lodgings—ou...
3,"('JQADiaries-v49-1825-01-p795.xml',)",jqadiaries-v49-1825-01-04,1825-01-04,NaN,4. VI:30. W. Findlay here; Statesman Newspaper...
4,"('JQADiaries-v49-1825-01-p795.xml',)",jqadiaries-v49-1825-01-05,1825-01-05,NaN,5. V:15. Mills came and took the draft of the ...
...,...,...,...,...,...
2184,"('JQADiaries-v31-1821-02-p508.xml',)",jqadiaries-v31-1821-02-25,1821-02-25,"Elections, Presidential (1824),Religion","25. VII: Attended Church at the Bath-room, and..."
2185,"('JQADiaries-v31-1821-02-p508.xml',)",jqadiaries-v31-1821-02-26,1821-02-26,"Adams-Onis Treaty,Elections, Presidential (182...",26. IV:15. The pressure of business upon me pu...
2186,"('JQADiaries-v31-1821-02-p508.xml',)",jqadiaries-v31-1821-02-27,1821-02-27,Caucus System for Nominating Presidential Cand...,27. VII:Mr Baker the British Consul General ca...
2187,"('JQADiaries-v31-1821-02-p508.xml',)",jqadiaries-v31-1821-02-28,1821-02-28,"Adams-Onis Treaty,Anti-Slavery Movements,Misso...",28. VII:Mr Ninian Edwards the Senator from Ill...
